# Create and run a local RAG pipeline from scratch


## Intro


### What is RAG?

- Retrieval - Find relevant info given a query
- Augmented - Take relevant info and augment our input (prompt) to an LLM with that relevant info
- Generation - Take the first two steps and pass them to an LLM for generative outputs


### Why RAG?

Improve generation outputs of LLMS

1. Prevents hallucinations - good looking text that is not necessarily factual
2. Work with custom data not internet-scale data


### What can RAG be used for?

1. Customer support Q&A chat
2. Email chain analysis
3. Company internal documentation chat
4. Textbook Q&A


### Why local?

1. Privacy - private documentation that you don't want to send to an API
2. Speed - no need to send data across the internet
3. Cost - No cost if using own hardware


### What we're going to build

- We're going to build RAG pipeline which enables us to chat with a PDF document, specifically an open-source nutrition textbook, ~1200 pages long.

- We'll write the code to:

1. Open a PDF document (you could use almost any PDF here).
2. Format the text of the PDF textbook ready for an embedding model (this process is known as text splitting/chunking).
3. Embed all of the chunks of text in the textbook and turn them into numerical representation which we can store for later.
4. Build a retrieval system that uses vector search to find relevant chunks of text based on a query.
5. Create a prompt that incorporates the retrieved pieces of text.
   Generate an answer to a query based on passages from the textbook.


## 1. Document/text processing and embedding creation


### Import and open PDF


In [39]:
# Import PDF

import os
import requests

# Get pdf document path
pdf_path = "human-nutrition-text.pdf"

# Download PDF
if not os.path.exists(pdf_path):
    print(f"[INFO] file doesn't exist, downloading...")

    # Enter the URL of the pdf
    url = "https://pressbooks.oer.hawaii.edu/humannutrition/open/download?type=pdf"

    # The local file name to save downloaded file
    filename = pdf_path

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open the file and save it
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"[INFO] The file has been downloaded and saved as{filename}")
    else:
        print(f"[INFO] Failed to download the file. Status code : {response.status_code}")

else:
    print(f"[INFO] File {pdf_path} exists")

[INFO] File human-nutrition-text.pdf exists


In [40]:
# Open PDF
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    """Performs minor formatting on text"""
    cleaned_text = text.replace("\n", " ").strip()

    # More text formatting functions can go in here
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text= page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({"page_number": page_number - 17,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

50it [00:00, 500.00it/s]

667it [00:01, 585.28it/s]


[{'page_number': -17,
  'page_char_count': 15,
  'page_word_count': 2,
  'page_sentence_count_raw': 1,
  'page_token_count': 3.75,
  'text': 'Human Nutrition'},
 {'page_number': -16,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [41]:
import random

random.sample(pages_and_texts, k=3)

[{'page_number': 396,
  'page_char_count': 1768,
  'page_word_count': 293,
  'page_sentence_count_raw': 17,
  'page_token_count': 442.0,
  'text': 'Iron Red blood cells contain the oxygen-carrier protein hemoglobin. It is composed of four globular peptides, each containing a heme complex. In the center of each heme, lies iron (Figure 11.2). Iron is needed for the production of other iron- containing proteins such as myoglobin. Myoglobin is a protein found in the muscle tissues that enhances the amount of available oxygen for muscle contraction. Iron is also a key component of hundreds of metabolic enzymes. Many of the proteins of the electron-transport chain contain iron–sulfur clusters involved in the transfer of high-energy electrons and ultimately ATP synthesis. Iron is also involved in numerous metabolic reactions that take place mainly in the liver and detoxify harmful substances. Moreover, iron is required for DNA synthesis. The great majority of iron used in the body is that rec

In [42]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-17,15,2,1,3.75,Human Nutrition
1,-16,0,1,1,0.00,
2,-15,188,26,1,47.00,Human Nutrition UNIVERSITY OF HAWAI‘I AT MĀNOA...
3,-14,607,100,5,151.75,Human Nutrition by University of Hawai‘i at Mā...
4,-13,827,130,4,206.75,Contents Preface xi About the Contributors xii...
